In [4]:
import asyncio
from datetime import datetime, timedelta
import pytz
from telethon import TelegramClient
from telethon.sessions import SQLiteSession
from telethon.tl.patched import Message
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Read values from .env
API_ID = os.getenv('API_ID')  # Your API_ID from .env
API_HASH = os.getenv('API_HASH')  # Your API_HASH from .env
SESSION_NAME = os.getenv('SESSION_NAME')  # Your session name from .env

# List of Telegram channel usernames
CHANNELS = [
    "From_hebron",
]

# Custom SQLiteSession with a longer timeout to prevent database lock issues
class CustomSQLiteSession(SQLiteSession):
    def __init__(self, session_id):
        super().__init__(session_id)
        # Set a custom timeout for SQLite database operations (in milliseconds)
        self._conn.execute("PRAGMA busy_timeout = 30000")  # 30 seconds timeout

class TelegramService:
    def __init__(self):
        """
        Initializes the TelegramService with API credentials using custom SQLite session.
        """
        self._client = TelegramClient(CustomSQLiteSession(SESSION_NAME), API_ID, API_HASH)

    async def read_messages_from_channel(self, channel_username: str, limit: int = 100, interval: int = 60) -> list[Message]:
        """
        Reads messages from a specified Telegram channel within the given time interval.

        :param channel_username: Username of the channel to read messages from.
        :param limit: Number of messages to fetch from the channel.
        :param interval: Time interval in minutes to filter messages.
        :return: List of filtered message objects.
        """
        try:
            await self._client.connect()
            if not await self._client.is_user_authorized():
                await self._client.start()

            channel = await self._client.get_entity(channel_username)
            messages = await self._client.get_messages(channel, limit=limit)

            # Current UTC time
            current_time = datetime.utcnow().replace(tzinfo=pytz.UTC)
            # Threshold time: messages from the last 'interval' minutes (in this case, 60 minutes)
            threshold_time = current_time - timedelta(minutes=interval)

            # Filter messages within the last 'interval' minutes
            last_interval_messages = [
                m for m in messages if m.date and m.date >= threshold_time
            ]

            return last_interval_messages
        except Exception as e:
            print(f"Error reading messages from channel {channel_username}: {e}")
            return []

    async def fetch_messages_from_channels(self):
        """
        Fetches messages from multiple Telegram channels and prints them.
        """
        messages: list[Message] = []
        await self._client.start()
        async with self._client:
            for channel in CHANNELS:
                # Fetch messages from each channel for the last 60 minutes
                messages += await self.read_messages_from_channel(
                    channel_username=channel,
                    limit=100,
                    interval=60,  # Last 60 minutes
                )

        await self.disconnect()

        # Print the filtered messages
        for m in messages:
            print(f"[{m.date}] {m.message}")  # Printing date and message content

        return messages

    async def disconnect(self):
        """
        Disconnects the Telegram client.
        """
        await self._client.disconnect()


# Check if there's already a running event loop
async def main():
    telegram_service = TelegramService()
    await telegram_service.fetch_messages_from_channels()

if __name__ == '__main__':
    try:
        # If there's an active event loop, run the main function with it
        loop = asyncio.get_running_loop()
        loop.create_task(main())
    except RuntimeError:
        # If no event loop is running, use asyncio.run()
        asyncio.run(main())


Task exception was never retrieved
future: <Task finished name='Task-6' coro=<main() done, defined at /tmp/ipykernel_17515/3071761887.py:100> exception=OperationalError('unable to open database file')>
Traceback (most recent call last):
  File "/tmp/ipykernel_17515/3071761887.py", line 101, in main
    telegram_service = TelegramService()
  File "/tmp/ipykernel_17515/3071761887.py", line 35, in __init__
    self._client = TelegramClient(CustomSQLiteSession(SESSION_NAME), API_ID, API_HASH)
  File "/tmp/ipykernel_17515/3071761887.py", line 26, in __init__
    super().__init__(session_id)
  File "/home/ofir.linux/.local/lib/python3.10/site-packages/telethon/sessions/sqlite.py", line 71, in __init__
    self._create_table(
  File "/home/ofir.linux/.local/lib/python3.10/site-packages/telethon/sessions/sqlite.py", line 162, in _create_table
    c.execute('create table {}'.format(definition))
sqlite3.OperationalError: unable to open database file
